In [1]:
import pandas as pd
#Source: https://www.seai.ie/data-and-insights/seai-statistics/monthly-energy-data/electricity/
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
from statsmodels.tsa.statespace.sarimax import SARIMAX
from plotly.subplots import make_subplots
import numpy as np

ModuleNotFoundError: No module named 'plotly'

In [2]:
total_generation_data = pd.read_csv('Ireland_Electricity_Monthly_Transposed.csv', engine='python', header=None)
total_generation_data.columns = total_generation_data.iloc[0]  # Set the first row as the header
total_generation_data = total_generation_data[1:]  # Remove the first row
total_generation_data['Month_Year'] = pd.to_datetime(total_generation_data['Month_Year'], format='%b-%y')

FileNotFoundError: [Errno 2] No such file or directory: 'Ireland_Electricity_Monthly_Transposed.csv'

In [ ]:
# Extract year and month for seasonality analysis
total_generation_data = total_generation_data[['Month_Year','Total Generation','Hydro','Wind','Solar']]

total_generation_data['Total Generation'] = pd.to_numeric(total_generation_data['Total Generation'], errors='coerce')
total_generation_data['Hydro'] = pd.to_numeric(total_generation_data['Hydro'], errors='coerce')
total_generation_data['Wind'] = pd.to_numeric(total_generation_data['Wind'], errors='coerce')
total_generation_data['Solar'] = pd.to_numeric(total_generation_data['Solar'], errors='coerce')


# Create a new column 'Total Renewables' by adding 'Hydro', 'Wind', and 'Solar'
total_generation_data['Total Renewables'] = (
    total_generation_data['Hydro'] +
    total_generation_data['Wind'] +
    total_generation_data['Solar']
)

# Calculate the percentage of renewable energy in total generation
total_generation_data['Percent Renewable'] = ((total_generation_data['Total Renewables'] / total_generation_data['Total Generation']) * 100).round(1)  # Round to 1 decimal place


In [ ]:
total_generation_data.columns

In [ ]:
total_generation_data.set_index('Month_Year', inplace=True)
total_generation_data.head()

In [ ]:
# Plotting Renewable % in Energy mix:

# Create a Plotly figure
fig = go.Figure()

# Add an area plot for 'Total Generation'
fig.add_trace(
    go.Scatter(
        x=total_generation_data.index,
        y=total_generation_data['Total Generation'],
        fill='tozeroy',  # Create the area effect
        mode='lines',  # Line plot with area filling
        name='Total Generation',
        line=dict(color='rgba(0, 100, 200, 0.5)')  # Adjust line color and transparency
    )
)

# Add an area plot for 'Wind'
fig.add_trace(
    go.Scatter(
        x=total_generation_data.index,
        y=total_generation_data['Wind'],
        fill='tozeroy',  # Create the area effect
        mode='lines',  # Line plot with area filling
        name='Wind',
        line=dict(color='rgba(0, 200, 100, 0.5)')  # Greenish color
    )
)

# Add an area plot for 'Solar'
fig.add_trace(
    go.Scatter(
        x=total_generation_data.index,
        y=total_generation_data['Solar'],
        fill='tozeroy',  # Create the area effect
        mode='lines',  # Line plot with area filling
        name='Solar',
        line=dict(color='rgba(255, 0, 0, 0.7)')  # Red color
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title='Total Electricity Generation, Wind, and Solar in Ireland (GWh)',
    xaxis=dict(
        title='Month/Year',
        showline=True,  # Show the x-axis line
        linecolor='black',  # Set the line color to black
        showgrid=False,  # Optionally hide grid lines
        mirror=True  # Optionally mirror x-axis
    ),
    yaxis=dict(
        title='Electricity Generated in GWh',
        showline=True,  # Show the y-axis line
        linecolor='black',  # Set the line color to black
        showgrid=False,  # Optionally hide grid lines
        mirror=True  # Optionally mirror y-axis
    ),
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white',  # Set the paper background to white
    showlegend=True  # Display legend for trace differentiation

)

# Show the area chart
fig.show()



In [ ]:
# Create a Plotly figure for 'Percent Renewable'
fig_percent_renewable = go.Figure()

# Add a line plot for 'Percent Renewable'
fig_percent_renewable.add_trace(
    go.Scatter(
        x=total_generation_data.index,
        y=total_generation_data['Percent Renewable'],
        mode='lines+markers',  # Line plot with markers
        name='Percent Renewable',
        line=dict(color='orange')  # Distinct color
    )
)

# Update layout to set the background to white and show x and y axes
fig_percent_renewable.update_layout(
    title='Percent Renewable of Total Electricity Generation Over Time in Ireland',
    xaxis=dict(
        title='Month/Year',
        showline=True,  # Show the x-axis line
        linecolor='black',  # Set the line color to black
        showgrid=False,  # Optionally hide grid lines
        mirror=True  # Optionally mirror x-axis
    ),
    yaxis=dict(
        title='Percent Renewable (%)',
        showline=True,  # Show the y-axis line
        linecolor='black',  # Set the line color to black
        showgrid=False,  # Optionally hide grid lines
        mirror=True  # Optionally mirror y-axis
    ),
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white',  # Set the paper background to white
    showlegend=True  # Display legend for trace differentiation
)

# Show the plot
fig_percent_renewable.show()

In [ ]:
population_data = pd.read_excel('Ireland Population.xlsx')
# Source: https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/PEA01/XLSX/2007/en

In [ ]:
population_data = population_data[['Year', 'Population']]  # Assuming these are the correct column names
population_data['Year'] = population_data['Year'].astype(int)

In [ ]:
# Create the primary line plot for total generation
# Create the primary line plot for 'Total Generation'
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=total_generation_data.index,
               y=total_generation_data['Total Generation'],
               name='Total Generation Historical Data')
)

# Create a secondary line plot for 'Population' on the same graph
fig.add_trace(
    go.Scatter(x=population_data['Year'],
               y=population_data['Population'],
               name='Population (in thousands)',
               yaxis='y2')
)

# Update layout to set white background and display axes
fig.update_layout(
    title='Total Monthly Electricity Generation and Population from 2010.(SARIMAX Forecast) ',
    xaxis=dict(
        title='Year',
        showline=True,  # Show x-axis line
        linecolor='black',  # Set the line color
        showgrid=True  # Display grid lines
    ),
    yaxis=dict(
        title='Total Generation Monthly in GWh',
        showline=True,  # Show y-axis line
        linecolor='black',  # Set the line color
        showgrid=True  # Display grid lines
    ),
    yaxis2=dict(
        title='Population (in thousands)',
        overlaying='y',
        side='right',
        showline=True,  # Show secondary y-axis
        linecolor='black',  # Set the line color
        showgrid=False  # Hide grid lines for secondary y-axis
    ),
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white',  # Set the paper background to white
    showlegend=True,  # Show legend for trace differentiation


    width=940,  # Increase plot width
    height=600,  # Increase plot height
    legend=dict(
        x=0.15,  # Center the legend horizontally
        y=-0.1,  # Place the legend below the plot
        orientation='h'  # Horizontal orientation
    )
)

# Fit the SARIMAX model (using SARIMAX)
model = SARIMAX(
    total_generation_data['Total Generation'],
    order=(1, 1, 1),  # ARIMA parameters (p, d, q)
    seasonal_order=(1, 1, 1, 12)  # SARIMA parameters (P, D, Q, s)
)

# Fit the model and forecast
results = model.fit()
forecast = results.forecast(steps=36)  # Forecast for 36 months ahead

# Create a future timeline for plotting
future_dates = pd.date_range(start=total_generation_data.index[-1], periods=36, freq='M')[1:]

# Add the SARIMAX forecast to the plot
fig.add_trace(
    go.Scatter(x=future_dates,
               y=forecast,
               name='SARIMAX Forecast',
               line=dict(dash='dash', color='orange'))
)

# Show the plot
fig.show()

In [ ]:
yearly_avg_gen = total_generation_data.groupby(total_generation_data.index.year).mean()
yearly_avg_gen.head()

In [ ]:
# Merge population data and yearly average generation data on 'Year'
combined_data = pd.merge(population_data, yearly_avg_gen, left_on='Year', right_index=True)

In [ ]:
# Calculate year-on-year percentage change for Population
combined_data['Population_YoY'] = combined_data['Population'].pct_change() * 100  # Convert to percentage

# Calculate year-on-year percentage change for Total Generation
combined_data['Total_Generation_YoY'] = combined_data[combined_data.columns[2]].pct_change() * 100  # Convert to percentage

# Create a scatter plot to show the relationship between year-on-year growth rates
fig = px.scatter(
    combined_data,
    x='Population_YoY',  # X-axis values
    y='Total_Generation_YoY',  # Y-axis values
    text='Year',  # Text label to display the year
    title='Year-on-Year Growth in Population and Total Electricity Generation YoY Growth',
    labels={'x': 'Population YoY (%)', 'y': 'Total Generation YoY (%)'}  # Axis labels
)

# Adding labels to points for better identification
fig.update_traces(textposition='top center')

# Add a horizontal line at 'y=0'
fig.add_shape(
    go.layout.Shape(
        type='line',
        x0=0,  # Start of the line
        y0=0,  # Y-position of the line
        x1=2,  # End of the line (adjust as needed)
        y1=0,  # Horizontal line
        line=dict(color='black', width=2)  # Line color and width
    )
)

# Update layout to move x-axis to 'y=0'
fig.update_layout(
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white',  # Set the paper background to white
    xaxis=dict(
        title='Population YoY (%)',
        showline=True,  # Show the x-axis line
        linecolor='black',  # Set the line color
        anchor='y',  # Anchors the x-axis to the 'y=0' position
        zeroline=True,  # Show zero line
        zerolinecolor='black',  # Set zero line color
        zerolinewidth=2  # Set zero line width
    ),
    yaxis=dict(
        title='Total Electricity Generation YoY (%)',
        showline=True,  # Show the y-axis line
        linecolor='black',  # Set the line color
        showgrid=True,  # Display grid lines for y-axis
        range=[-4, 4],  # Adjust y-axis range
        zeroline=True,  # Show zero line
        zerolinecolor='black',  # Set zero line color
        zerolinewidth=2,  # Set zero line width
        fixedrange=False,  # Allow zooming on y-axis
    ),
    height=660,  # Adjust plot height
    width=920,  # Adjust plot width
)

In [ ]:
combined_data.head()

In [ ]:
combined_data.columns

In [ ]:
combined_data.tail()

In [ ]:
# Remove rows with NaNs in 'Population_YoY' and 'Total_Generation_YoY'
combined_data.dropna(subset=['Population_YoY', 'Total_Generation_YoY'], inplace=True)

# Calculate the Pearson correlation between 'Population_YoY' and 'Total_Generation_YoY'
correlation, p_value = stats.pearsonr(combined_data['Population_YoY'], combined_data['Total_Generation_YoY'])

print("Correlation between Population YoY and Total Generation YoY:", correlation)
print("P-value for significance:", p_value)


In [ ]:
# Develop a linear regression model to predict 'Total_Generation_YoY' based on 'Population_YoY'
X = combined_data[['Population_YoY']]  # Predictor variable
y = combined_data['Total_Generation_YoY']  # Target variable

# Ensure there are no NaNs in X and y before fitting the regression model
if X.isnull().any().any() or y.isnull().any():
    raise ValueError("Input data contains NaNs or infinite values.")

regressor = LinearRegression()
regressor.fit(X, y)  # Fit the linear regression model

# Coefficients for the linear regression model
print("Linear Regression Coefficients:", regressor.coef_, "Intercept:", regressor.intercept_)

# Predict 'Total Generation YoY' for 2024 and 2025 with a given population growth rate
population_growth_rate = 2  # 2% growth rate

# Create a DataFrame for prediction
prediction_df = pd.DataFrame({
    'Year': [2024, 2025],
    'Population_YoY': [population_growth_rate, population_growth_rate]
})

# Predict the 'Total_Generation_YoY' for the given years
prediction_df['Total_Generation_YoY'] = regressor.predict(prediction_df[['Population_YoY']])

# Calculate 'Total Generation' for 2024 and 2025 based on predicted 'Total_Generation_YoY'
last_total_generation = combined_data['Total Generation'].iloc[-1]  # Last known value

# Calculate 'Total_Generation' for 2024 based on predicted YoY change
prediction_df.loc[prediction_df['Year'] == 2024, 'Total_Generation'] = last_total_generation * (
    1 + prediction_df.loc[prediction_df['Year'] == 2024, 'Total_Generation_YoY'].values[0] / 100
)

# Calculate 'Total_Generation' for 2025 based on the 2024 calculated value
base_2024 = prediction_df.loc[prediction_df['Year'] == 2024, 'Total_Generation'].values[0]
prediction_df.loc[prediction_df['Year'] == 2025, 'Total_Generation'] = base_2024 * (
    1 + prediction_df.loc[prediction_df['Year'] == 2025, 'Total_Generation_YoY'].values[0] / 100
)

print("Predicted Total Generation for 2024 and 2025:")
print(prediction_df[['Year', 'Total_Generation']])

In [ ]:
# Create the Plotly figure
# Create the Plotly figure
fig = go.Figure()

# Add historical total generation as a solid line
fig.add_trace(go.Scatter(x=combined_data['Year'],
                         y=combined_data['Total Generation'],
                         mode='lines',
                         name='Total Yearly Average Generation Historical'))

# Connect the last historical point to the first predicted point with a dashed line
connect_point_x = [combined_data['Year'].iloc[-1], prediction_df['Year'].iloc[0]]
connect_point_y = [combined_data['Total Generation'].iloc[-1], prediction_df['Total_Generation'].iloc[0]]

fig.add_trace(go.Scatter(x=connect_point_x,
                         y=connect_point_y,
                         mode='lines+markers',
                         showlegend=False,  # Hide legend for this trace
                         line=dict(dash='dash', color='orange')))

# Add predicted total generation for 2024 and 2025
fig.add_trace(go.Scatter(x=prediction_df['Year'],
                         y=prediction_df['Total_Generation'],
                         mode='lines+markers',
                         name='Total Generation Predicted',
                         line=dict(dash='dash', color='orange')))

# Update the layout to achieve your requirements
fig.update_layout(
    title='Total Yearly Average Electricity Generation (GWh): Historical and Predicted (2024-2025)',
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set paper background to white
    legend=dict(
        orientation='h',  # Horizontal orientation
        x=0,  # Align legend to the left
        y=-0.1  # Position legend at the bottom
    ),
    xaxis=dict(
        title='Year',
        showline=True,  # Display x-axis line
        linewidth=2,  # Set x-axis line width
        linecolor='black',  # Set x-axis line color
    ),
    yaxis=dict(
        title='Total Annual Average Electicity Generation (GWh)',
        showline=True,  # Display y-axis line
        linewidth=2,  # Set y-axis line width
        linecolor='black',  # Set y-axis line color
    ),
    height=600,  # Set plot height
    width=800  # Set plot width
)

# Show the plot
fig.show()